In [4]:
# @title ワイド⇒ロングフォーマット化
# Colab用：ロングフォーマット変換 + ラベル化（文字列化）エクスポート対応（自動判定版）
# 使い方：
# 1) セルを実行 → アップロードウィジェットにCSV/XLSXをドロップ/選択
# 2) （XLSXでシート複数の場合）キー列を選んでマージ
# 3) ラベル列は自動判定（アンダースコア2つ"_"を含まない列）※ID/Nameは除外固定
# 4) スコア列（試験名_時期_科目）ごとの満点を調整（初期値＝実測最大を上回る最小の100刻み）
# 5) 「ロングフォーマット化してCSV出力」を押す → ダウンロード開始

import io
import os
import re
import pandas as pd
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Layout

# --------------------------
# ウィジェットの準備
# --------------------------
uploader = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='ファイルを選択/ドロップ'
)

info_html = widgets.HTML(value="")
sheet_info_html = widgets.HTML(value="")
keycol_dd = widgets.Dropdown(description='キー列', options=[], disabled=True)
merge_btn = widgets.Button(description='シートをマージ', button_style='', disabled=True)
label_cols = widgets.SelectMultiple(description='ラベル（自動判定・表示のみ）', options=[], rows=10, disabled=True)
score_cols_box = widgets.VBox([])  # スコア列ごとの満点入力を置く
convert_btn = widgets.Button(description='ロングフォーマット化してCSV出力', button_style='primary', disabled=True)
preview_out = widgets.Output()

# 内部状態
state = {
    "df": None,
    "long_df": None,
    "test_score_cols": [],
    "max_inputs": {},      # {col: widgets.FloatText}
    "uploaded_name": None,
    "sheets": None,        # dict of sheet_name -> DataFrame (XLSXのみ)
}

# --------------------------
# ユーティリティ
# --------------------------
def read_csv_with_encodings(bts: bytes, name: str) -> pd.DataFrame:
    tried = ["utf-8", "shift_jis", "cp932"]
    last_err = None
    for enc in tried:
        try:
            return pd.read_csv(io.BytesIO(bts), encoding=enc)
        except UnicodeDecodeError as e:
            last_err = e
            continue
    raise UnicodeDecodeError(f"CSV読み込み失敗: {name}. 試行済みエンコーディング: {tried}", b'', 0, 1, "decode error")

def build_test_score_list(df: pd.DataFrame):
    # 列名が「xxx_yyy_zzz」（アンダースコア2つ＝3要素）のものをスコア列とみなす
    pat = re.compile(r'^[^_]+_[^_]+_[^_]+$')
    valid, invalid = [], []
    for col in df.columns:
        if pat.match(col):
            valid.append(col)
        else:
            invalid.append(col)
    return valid, invalid

def show_overview(df: pd.DataFrame):
    miss = "<br>".join([f"{c}: {int(df[c].isna().sum())}" for c in df.columns])
    info_html.value = (
        f"<b>データ次元:</b> {df.shape[0]} 行 x {df.shape[1]} 列<br>"
        f"<b>各列の欠損数:</b><br>{miss}"
    )

def setup_score_inputs(df: pd.DataFrame, test_cols):
    import math  # 初期値の丸めに使用

    state["max_inputs"].clear()
    rows = []

    header = widgets.HTML(
        value=(
            "<b>スコア列の満点設定（初期値＝実測最大値を“上回る”最小の100刻み）</b><br>"
            "<span style='font-size:12px;color:#666;'>例: 78→100, 100→200, 280→300。"
            "列名は折返し、実測最大値はコンパクト表示です。</span>"
        )
    )

    for col in test_cols:
        # 実測最大値
        actual = pd.to_numeric(df[col], errors='coerce').max()
        if pd.isna(actual):
            actual_str = "-"
            init_full_score = 100.0
        else:
            # 実測最大値より“厳密に大きい”最小の100刻み
            # 例: 78→100, 100→200, 280→300
            init_full_score = (math.floor(float(actual) / 100.0) + 1) * 100
            actual_str = str(int(actual)) if float(actual).is_integer() else str(round(float(actual), 1))

        # 1) 列名表示
        name = widgets.HTML(
            value=f"<div title='{col}' "
                  f"style='max-width:420px; white-space:normal; word-break:break-word; line-height:1.2;'>{col}</div>",
            layout=Layout(width='50%')
        )

        # 2) 満点入力（初期値を上記ルールで設定）
        ft = widgets.FloatText(
            value=float(init_full_score),
            layout=Layout(width='120px')
        )

        # 3) 実測最大値ヒント
        hint = widgets.HTML(
            value=f"<div style='font-size:12px; color:#555; white-space:nowrap;'>max: {actual_str}</div>",
            layout=Layout(width='90px')
        )

        state["max_inputs"][col] = ft

        row = widgets.HBox(
            [name, ft, hint],
            layout=Layout(align_items='center')
        )
        rows.append(row)

    score_cols_box.children = [header] + (rows if rows else [widgets.HTML(
        value="<i>スコア列（試験名_時期_科目）の列は見つかりませんでした。</i>"
    )])

PROTECTED_LABEL_IGNORE = {"ID", "Name"}

def get_auto_label_cols(df: pd.DataFrame, test_cols):
    # スコア列以外（=アンダースコア2つでない列）を自動でラベル候補に（ID/Nameは除外）
    return [c for c in df.columns if c not in test_cols and c not in PROTECTED_LABEL_IGNORE]

def enable_label_select(df: pd.DataFrame):
    # 自動計算：スコア列以外をラベルに。UIは表示のみ（無効化）
    auto_labels = get_auto_label_cols(df, state["test_score_cols"])
    label_cols.options = auto_labels
    label_cols.value = tuple(auto_labels)
    label_cols.disabled = True

def do_labelize(df: pd.DataFrame, label_selection):
    """
    label_selection に含まれる列を「ラベル（文字列）」として扱う。
    数値として入力されている値は CSV で文字列として解釈されるよう、先頭に ' を付ける。
    既に ' で始まる値はそのまま。NaN は空文字にする。
    """
    import re
    df2 = df.copy()
    sel = set(label_selection)

    def stringify_with_apostrophe(series: pd.Series) -> pd.Series:
        is_series_numeric = pd.api.types.is_numeric_dtype(series.dtype)

        def transform(v):
            if pd.isna(v):
                return ""  # 欠損は空文字で出力
            s = str(v)
            if s.startswith("'"):
                return s  # 既に ' 付きはそのまま

            # もともと数値型の列は必ず ' を付与
            if is_series_numeric:
                return "'" + s

            # 文字列型でも数値っぽい表記なら ' を付与（例: "123", "3.14", "1,234.5"）
            if re.fullmatch(r"\s*-?[\d,]+(\.\d+)?\s*", s):
                return "'" + s.strip()

            return s

        return series.map(transform).astype(str)

    for c in df2.columns:
        if c in sel and c not in PROTECTED_LABEL_IGNORE:
            df2[c] = stringify_with_apostrophe(df2[c])
        else:
            # ラベル化しない列は、従来通り：可能なら数値化（安全に）
            df2[c] = pd.to_numeric(df2[c], errors='ignore')

    return df2

def melt_long(df_wide: pd.DataFrame, test_cols, max_map):
    """
    ・ロング化したときに '元スコア' と 'スコア(%)' を内部では保持
    ・最終的に 'スコア' 列のみを出力する
    """

    id_vars = [c for c in df_wide.columns if c not in test_cols]
    long_df = pd.melt(
        df_wide,
        id_vars=id_vars,
        value_vars=test_cols,
        var_name="試験情報",
        value_name="スコア"
    )

    # 元スコアを別列に保持（内部処理用）
    long_df["元スコア"] = long_df["スコア"].astype(str)

    # 数値化を試みる
    num = pd.to_numeric(long_df["スコア"], errors='coerce')

    # 百分率を計算
    def to_percent(row):
        x = row["__num__"]
        if pd.isna(x):
            return pd.NA
        m = float(max_map.get(row["試験情報"], 100.0))
        if m == 0:
            return pd.NA
        perc = round((float(x) / m) * 100, 1)
        return int(perc) if float(perc).is_integer() else perc

    long_df["__num__"] = num
    long_df["スコア(%)"] = long_df.apply(to_percent, axis=1)
    long_df = long_df.drop(columns="__num__")

    # 試験情報を分割
    split_cols = long_df["試験情報"].str.split("_", n=2, expand=True)
    if split_cols.shape[1] != 3:
        raise ValueError("テストスコア列名の形式が正しくありません（xxx_yyy_zzz）。")
    split_cols.columns = ["試験名", "時期", "科目"]

    # 最終出力用の「スコア」を決定：
    # ・数値化できたものは「スコア(%)」
    # ・非数値だったものは「元スコア」
    long_df["スコア"] = long_df["スコア(%)"].combine_first(long_df["元スコア"])

    # 最終的には 'スコア' 列のみを残す
    long_df = pd.concat(
        [long_df[id_vars], split_cols, long_df[["スコア"]]],
        axis=1
    )

    return long_df

def download_csv(df: pd.DataFrame, base_name="long_output.csv"):
    path = base_name
    df.to_csv(path, index=False, encoding='utf-8-sig')
    files.download(path)

# --------------------------
# ハンドラ
# --------------------------
def on_upload_change(change):
    if len(uploader.value) == 0:
        return
    preview_out.clear_output()
    info_html.value = ""
    sheet_info_html.value = ""
    keycol_dd.options = []
    keycol_dd.disabled = True
    merge_btn.disabled = True
    label_cols.options = []
    label_cols.disabled = True
    score_cols_box.children = []
    convert_btn.disabled = True
    state.update({"df": None, "long_df": None, "test_score_cols": [], "max_inputs": {}, "uploaded_name": None, "sheets": None})

    # 受け取り
    item = list(uploader.value.values())[0]
    name = item['metadata']['name']
    bts = item['content']
    state["uploaded_name"] = name

    try:
        if name.lower().endswith('.csv'):
            df = read_csv_with_encodings(bts, name)
            state["df"] = df
            show_overview(df)
            valid, invalid = build_test_score_list(df)
            state["test_score_cols"] = valid
            setup_score_inputs(df, valid)
            enable_label_select(df)  # 自動ラベル表示
            convert_btn.disabled = False
            sheet_info_html.value = "<i>CSVを読み込みました。</i>"
        elif name.lower().endswith('.xlsx'):
            sheets = pd.read_excel(io.BytesIO(bts), sheet_name=None)
            state["sheets"] = sheets
            if len(sheets) == 0:
                raise ValueError("XLSXにシートがありません。")
            elif len(sheets) == 1:
                df = list(sheets.values())[0]
                state["df"] = df
                show_overview(df)
                valid, invalid = build_test_score_list(df)
                state["test_score_cols"] = valid
                setup_score_inputs(df, valid)
                enable_label_select(df)  # 自動ラベル表示
                convert_btn.disabled = False
                sheet_info_html.value = "<i>1シートのみ検出。マージは不要です。</i>"
            else:
                # 複数シート → キー列選択して左結合でマージ
                # 全シートの共通列を候補にする
                common_cols = None
                for df in sheets.values():
                    if common_cols is None:
                        common_cols = set(df.columns)
                    else:
                        common_cols &= set(df.columns)
                common_cols = sorted(list(common_cols)) if common_cols else []
                if not common_cols:
                    # 共通がなければ最初のシートの列を候補に
                    first_cols = list(list(sheets.values())[0].columns)
                    keycol_dd.options = first_cols
                else:
                    keycol_dd.options = common_cols

                keycol_dd.disabled = False
                merge_btn.disabled = False

                sheet_names = ", ".join(sheets.keys())
                sheet_info_html.value = (
                    f"<b>複数シートを検出:</b> {sheet_names}<br>"
                    "キー列を選択して「シートをマージ」を押してください（1枚目を基準に左結合）。"
                )
        else:
            sheet_info_html.value = "<span style='color:red'>CSVまたはXLSXをアップロードしてください。</span>"
    except Exception as e:
        sheet_info_html.value = f"<span style='color:red'>読み込みエラー: {e}</span>"

def on_merge_clicked(btn):
    sheets = state["sheets"]
    if not sheets:
        return
    key = keycol_dd.value
    if not key:
        sheet_info_html.value = "<span style='color:red'>キー列を選択してください。</span>"
        return

    # 1枚目を基準に、以降を左結合（新規列のみ追加）
    base_name = list(sheets.keys())[0]
    base_df = sheets[base_name].copy()
    for sn, df in list(sheets.items())[1:]:
        if key not in df.columns:
            continue
        new_cols = [c for c in df.columns if c != key and c not in base_df.columns]
        if new_cols:
            base_df = base_df.merge(df[[key] + new_cols], on=key, how='left')

    state["df"] = base_df
    show_overview(base_df)
    valid, invalid = build_test_score_list(base_df)
    state["test_score_cols"] = valid
    setup_score_inputs(base_df, valid)
    enable_label_select(base_df)  # 自動ラベル表示
    convert_btn.disabled = False
    sheet_info_html.value = "<i>マージ完了。</i>"

def on_convert_clicked(btn):
    preview_out.clear_output()
    df = state["df"]
    if df is None:
        with preview_out:
            print("エラー：データが読み込まれていません。")
        return

    # 自動ラベル列を使用（UI選択値ではなく自動判定結果を用いる）
    auto_labels = get_auto_label_cols(df, state["test_score_cols"])
    labeled_df = do_labelize(df, auto_labels)

    # スコア列の満点取得（ある場合のみ使う）
    max_map = {}
    for col in state["test_score_cols"]:
        try:
            max_map[col] = float(state["max_inputs"][col].value)
        except Exception:
            max_map[col] = 100.0

    try:
        if not state["test_score_cols"]:
            # ★ スコア列が無い場合：そのまま“カテゴリ列のみ”のロング（=ワイドのまま）を出力
            long_df = labeled_df.copy()
            state["long_df"] = long_df
            with preview_out:
                print("スコア列（xxx_yyy_zzz）が見つからなかったため、スコア無しで出力します。")
                print(f"出力データの次元: {long_df.shape[0]} 行 x {long_df.shape[1]} 列")
                display(long_df.head(20))
                print("CSVを書き出しています（utf-8-sig）...")
            base = os.path.splitext(state["uploaded_name"] or "output")[0]
            outname = f"{base}_long.csv"
            download_csv(long_df, outname)
            return

        # ★ スコア列あり：通常どおりロング化して出力
        long_df = melt_long(labeled_df, state["test_score_cols"], max_map)
        state["long_df"] = long_df

        with preview_out:
            print(f"ロングフォーマット後の次元: {long_df.shape[0]} 行 x {long_df.shape[1]} 列")
            display(long_df.head(20))
            print("CSVを書き出しています（utf-8-sig）...")
        base = os.path.splitext(state["uploaded_name"] or "output")[0]
        outname = f"{base}_long.csv"
        download_csv(long_df, outname)
    except Exception as e:
        with preview_out:
            print(f"エラー: {e}")

# --------------------------
# レイアウト表示
# --------------------------
uploader.observe(on_upload_change, names='value')
merge_btn.on_click(on_merge_clicked)
convert_btn.on_click(on_convert_clicked)

label_caption = widgets.HTML(
    "<b>列をラベル（文字列）として扱う（自動判定）</b><br>"
    "<span style='font-size:12px;color:#666;'>アンダースコア2つの列のみスコア列／その他は自動でラベル化します（ID/Nameは除外）。</span>"
)

display(widgets.VBox([
    widgets.HTML("<h3>データロングフォーマット変換（Colab版）＋ ラベル化エクスポート</h3>"),
    uploader,
    sheet_info_html,
    widgets.HBox([keycol_dd, merge_btn]),
    info_html,
    widgets.HTML("<hr>"),
    label_caption,
    label_cols,  # 表示のみ（自動判定結果の確認用）
    widgets.HTML("<hr>"),
    score_cols_box,
    widgets.HTML("<hr>"),
    convert_btn,
    preview_out
]))


/tmp/ipython-input-3071445963.py:186: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2[c] = pd.to_numeric(df2[c], errors='ignore')
/tmp/ipython-input-3071445963.py:186: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2[c] = pd.to_numeric(df2[c], errors='ignore')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 命名規則
「ID」「Name」 … 必須列。 \
数値データ(numeric data)の命名規則：「試験名_時期_科目」の３要素で命名（アンダースコア"_"が２つ含まれていることがトリガー）。中身には数値データだけを入力する。欠損値がある場合は空白にする。 \
カテゴリデータ(category data)の命名規則：数値データのフォーマットとは異なれば
（アンダースコアが２つ含まれていなければ）全てカテゴリデータと認識。数値データもテキストとみなす。